### 环境

In [ ]:
# !pip install pytorch 
# !pip install scikit-learn
# !pip install sklearn_extensions

In [ ]:
# !ln -s ~/Downloads/siblings_pose siblings_pose
# !ln -s ~/Downloads/siblings_face siblings_face

[example](https://zhuanlan.zhihu.com/p/82810188)

Q 视频的label是对视频整体的，但Feature却是逐帧的。

为了聚合帧级预测得到视频级预测结果，提出四种方式：

1）使用最后一个时间帧的预测结果为最终视频预测结果；√  

2）对所有时序帧的预测结果进行最大池化得到最终结果；

3）加和所有帧预测结果，并取最大值；

4）结合g对每帧预测结果进行线性加权，求和并返回最大值

对于简单分类器
- 实际处理中将视频的label复制到每个frame中    
- 如此问题转变为简单的分类问题


- **Face Feature**  
frame, face_id, timestamp, confidence, success，[709*features]

- 分别放入 每次放入一个person  
person * frame  * feature  
2      * frame  * 709  

- 共同放入  
frame * 1418  

### DATA

需要分别读入face和pose的数据并对应起来
- 对于siblings_face 需要读入json  
- 对于siblings_pose 需要读入csv    
- 对于Label 需要读入csv


*特别注意对于face，需要去除置信度小于阈值的数据  
*要将video的label扩展到每个frame

In [ ]:
import csv
import pandas as pd

In [4]:
import csv
face_feature_file = "face.csv" # note that this is only single video feat

with open("face.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    # for row in spamreader:
        # print(', '.join(row))

In [26]:
import pandas as pd
x_train = pd.read_csv("face.csv")
x_train.head(5)


,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.000,0.98,1,0.037185,0.380190,-0.924161,-0.258944,0.495936,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
1,1,1,0.000,0.73,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
2,1,2,0.000,0.03,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,2,0,0.033,0.98,1,0.043811,0.401678,-0.914732,-0.255940,0.517694,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,2,1,0.033,0.73,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0


In [27]:
from sklearn.model_selection import train_test_split
import numpy as np

x_train = x_train.to_numpy()
# print(x_train.shape) # 204 * 714
x = x_train[:,5:714] # x * 709
y = x_train[:,4] # x * 1
# batch unsqueeze
x = np.expand_dims(x, axis=0)
y = np.expand_dims(y, axis=0)

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

### LSTM

> 25 hidden units LSTM layer followed by 100 units dense layer with the rectified linear unit and 2 units dense layer with softmax for LSTM.  
>100 LSTM units for the first layer for LSTM


[torch](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)   
[mil](https://github.com/Marsrocky/Emotiw-Engagement-Prediction/blob/master/mil_lstm.py)  
[Engagement_Detection_OpenFace_Bi-LSTM](https://github.com/CopurOnur/Engagement_Detection_OpenFace_Bi-LSTM/blob/main/Engagement_Detection_OpenFace_Bi_LSTM_test.ipynb)  
[action unit](https://github.com/MRzzm/action-recognition-models-pytorch/tree/master/CNN%2BLSTM/ALSTM)  
为了聚合帧级预测得到视频级预测结果，提出四种方式：

1）使用最后一个时间帧的预测结果为最终视频预测结果；

2）对所有时序帧的预测结果进行最大池化得到最终结果；

3）加和所有帧预测结果，并取最大值；

4）结合g对每帧预测结果进行线性加权，求和并返回最大值

In [1]:
from torch import nn
import torch

class LSTM(nn.Module):
    def __init__(self,input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc1 = nn.Linear(100, 100)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(100, 2) # 2 classes
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.unsqueeze(1)
        out, _ = self.lstm(x) # 1 * 100
        out = self.fc1(out[:, -1, :])
        out = self.relu(out)
        out = self.fc2(out)
        out = self.softmax(out)
        return out

In [2]:
class mil_regression(nn.Module):
    def __init__(self, input_size=709, hidden_size=100, num_layers=1, output_size=2):
        ''' use LSTM for MIL '''
        super(mil_regression, self).__init__()
        self.net = LSTM(input_size, hidden_size, num_layers)
        self.class_num = output_size

    def forward(self, inputs):
        # input shape: (batch_num, frame_num, feature_size)

        self.b, self.seg_num, self.feature_num = inputs.shape #
        outputs = torch.zeros((self.b, self.seg_num, self.class_num)).cuda() # batch * frame * 2（bool）

        for i in range(self.seg_num):
            outputs[:,i,:] = self.net(inputs[:,i,:]).squeeze()

        # for idx, seg in enumerate(inputs):
        #     seg = Variable(seg).cuda()
        #     outputs[idx] = self.net(seg)

        # 视频特征 = frame取平均
        output = torch.mean(outputs, 1).cuda()
        return output

In [34]:
# dataset = OpenfaceDataset()
# train_loader = torch.utils.data.DataLoader(dataset, batch_size=1, num_workers=1, shuffle=True)
# for idx, (data, target) in enumerate(train_loader):
#     output = net(data)
#     print (output)
#     break
# 
import torch 

model = mil_regression().cuda()

x_train = torch.from_numpy(x,device="cuda") 
y_train = torch.from_numpy(y,device="cuda")

epochs = 150
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for i in range(epochs):
    for i in range(len(x_train)): # 每次处理一个视频(对batch)
        x = x_train[i:i+10]
        y = y_train[i:i+10]

        optimizer.zero_grad()

        y_pred = model(x)

        single_loss = loss_function(y_pred, y)
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

RuntimeError: Input and parameter tensors are not at the same device, found input tensor at cpu and parameter tensor at cuda:0

In [10]:
x = torch.tensor(x_train)
x = x.unsqueeze(0).to(torch.float)
print(x.shape) # [1, 153, 709]
y_pred = model(x.cuda())

torch.Size([1, 153, 709])


### Random Forest

> 50 trees and maximum depth of 50 for RF  
> 25 trees with maximum depths of 10 for RF

In [33]:



from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import 




# dt = DictVectorizer(sparse=False)

# print(x_train.to_dict(orient="record"))
# # 按行，样本名字为键，列名也为键，[{"1":1,"2":2,"3":3}]
# x_train = dt.fit_transform(x_train.to_dict(orient="record"))
# x_test = dt.fit_transform(x_test.to_dict(orient="record"))


# 使用随机森林预测一个frame
rfc = RandomForestClassifier(n_estimators=25, max_depth=10)
rfc.fit(x_train, y_train)
rfc_y_predict = rfc.predict(x_test)
print(rfc.score(x_test, y_test))
print(classification_report(y_test, rfc_y_predict, target_names=["died", "survived"]))

1.0
              precision    recall  f1-score   support

        died       1.00      1.00      1.00        14
    survived       1.00      1.00      1.00        37

    accuracy                           1.00        51
   macro avg       1.00      1.00      1.00        51
weighted avg       1.00      1.00      1.00        51



In [ ]:
from sklearn.ensemble import RandomForestClassifier 
 # 建立随机森林分类器
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# 建立决策树分类器
clf = DecisionTreeClassifier(random_state=0)

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# 建立KNN分类器
clf = KNeighborsClassifier(n_neighbors=3)

### SVM

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# 读取特征数据
data = pd.read_csv('feature.csv')

# 将one-hot标签合并为单个列
y = data.iloc[:, -4:].idxmax(axis=1)

# 分离特征和标签
X = data.iloc[:, :-4]

# 划分训练和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# 建立SVM分类器
clf = SVC(kernel='linear')

# 训练分类器
clf.fit(X_train, y_train)

# 评估分类器性能
accuracy = clf.score(X_test, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))


### ELM
[ELM sklearn](http://wdm0006.github.io/sklearn-extensions/extreme_learning_machines.html#usage)  
[ELM](https://github.com/5663015/elm)  
[ELM-pytorch](https://github.com/naderAsadi/ELM-Pytorch/blob/master/models.py)  


100 hidden units with radial basis function with 0.01 width as the activation function for ELM  
500 hidden units for ELM;  

In [39]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn_extensions.extreme_learning_machines.elm import GenELMClassifier

from sklearn_extensions.extreme_learning_machines.random_layer import RBFRandomLayer, MLPRandomLayer


def make_classifiers():
    """

    :return:
    """

    names = ["ELM(10,tanh)", "ELM(10,tanh,LR)", "ELM(10,sinsq)", "ELM(10,tribas)", "ELM(hardlim)", "ELM(20,rbf(0.1))"]

    nh = 10

    # pass user defined transfer func
    sinsq = (lambda x: np.power(np.sin(x), 2.0))
    srhl_sinsq = MLPRandomLayer(n_hidden=nh, activation_func=sinsq)

    # use internal transfer funcs
    srhl_tanh = MLPRandomLayer(n_hidden=nh, activation_func='tanh')
    srhl_tribas = MLPRandomLayer(n_hidden=nh, activation_func='tribas')
    srhl_hardlim = MLPRandomLayer(n_hidden=nh, activation_func='hardlim')

    # use gaussian RBF
    srhl_rbf = RBFRandomLayer(n_hidden=nh*2, rbf_width=0.1, random_state=0)
    log_reg = LogisticRegression()

    classifiers = [GenELMClassifier(hidden_layer=srhl_tanh),
                   GenELMClassifier(hidden_layer=srhl_tanh, regressor=log_reg),
                   GenELMClassifier(hidden_layer=srhl_sinsq),
                   GenELMClassifier(hidden_layer=srhl_tribas),
                   GenELMClassifier(hidden_layer=srhl_hardlim),
                   GenELMClassifier(hidden_layer=srhl_rbf)]

    return names, classifiers

# generate some datasets
names, classifiers = make_classifiers()

# iterate over datasets
for idx, ds in enumerate(datasets):
    # pre-process dataset, split into training and test part
    X, y = ds
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=0)
    y_test = y_test.reshape(-1, )
    y_train = y_train.reshape(-1, )

    # iterate over classifiers
    print('Dataset: %s' % (idx, ))
    for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        print('Model %s score: %s' % (name, score))


TypeError: __init__() takes 1 positional argument but 3 were given

pytorch version

In [1]:
import torch
import torch.nn as nn

###############
# ELM
###############
class ELM():
    def __init__(self, input_size, h_size, num_classes, device=None):
        self._input_size = input_size
        self._h_size = h_size
        self._output_size = num_classes
        self._device = device

        self._alpha = nn.init.uniform_(torch.empty(self._input_size, self._h_size, device=self._device), a=-1., b=1.)
        self._beta = nn.init.uniform_(torch.empty(self._h_size, self._output_size, device=self._device), a=-1., b=1.)

        self._bias = torch.zeros(self._h_size, device=self._device)

        self._activation = torch.sigmoid

    def predict(self, x):
        h = self._activation(torch.add(x.mm(self._alpha), self._bias))
        out = h.mm(self._beta)

        return out

    def fit(self, x, t):
        temp = x.mm(self._alpha)
        H = self._activation(torch.add(temp, self._bias))

        H_pinv = torch.pinverse(H)
        self._beta = H_pinv.mm(t)


    def evaluate(self, x, t):
        y_pred = self.predict(x)
        acc = torch.sum(torch.argmax(y_pred, dim=1) == torch.argmax(t, dim=1)).item() / len(t)
        return acc

#####################
# Helper Functions
#####################
def to_onehot(batch_size, num_classes, y, device):
    # One hot encoding buffer that you create out of the loop and just keep reusing
    y_onehot = torch.FloatTensor(batch_size, num_classes).to(device)
    #y = y.type(dtype=torch.long)
    y = torch.unsqueeze(y, dim=1)
    # In your for loop
    y_onehot.zero_()
    y_onehot.scatter_(1, y, 1)

    return y_onehot

### 结果测量
- F1 Score  
- Precision  
- Recall  


In [ ]:
from sklearn.metrics import classification_report

# 计算分类器性能指标
print(classification_report(y_test, y_pred))
